<h1>ANOVOS - Data Transformer<span class="tocSkip"></span></h1>
<p> Following notebook shows the list of functions related to "data transformer" module provided under ANOVOS package and how it can be invoked accordingly</p>
<div class="toc"><ul class="toc-item"><li><span><a href="#Attribute-Binning-(discretization)" data-toc-modified-id="Attribute-Binning-1">Attribute Binning (discretization)</a></span></li><li><span><a href="#Monotonic-Binning" data-toc-modified-id="Monotonic-Binning-2">Monotonic Binning</a></span></li><li><span><a href="#Categorical-Attribute-to-Numerical-Attribute-Conversion" data-toc-modified-id="Categorical-Attribute-to-Numerical-Attribute-Conversion-3">Categorical Attribute to Numerical Attribute Conversion</a></span></li><li><span><a href="#Missing-Value-Imputation" data-toc-modified-id="Missing-Value-Imputation">Missing Value Imputation</a></span></li><li><span><a href="#Outlier-Categories-Treatment" data-toc-modified-id="Outlier-Categories-Treatment-5">Outlier Categories Treatment</a></span></li></ul></div>

**Setting Spark Session**

In [1]:
from anovos.shared.spark import *

**Input/Output Path** 

In [2]:
inputPath = "../data/income_dataset/csv"
outputPath = "../output/income_dataset/data_transformer"

**Read Input Data** 

In [3]:
from anovos.data_ingest.data_ingest import read_dataset
from pyspark.sql import functions as F
df = read_dataset(file_path = inputPath, file_type = "csv",
                  file_configs = {"header": "True", "delimiter": "," , "inferSchema": "True"})
df.toPandas().head(5)

,ifa,age,workclass,fnlwgt,logfnl,empty,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,label
0,1a,NaN,State-gov,77516.0,4.889391,None,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,UnitedStates,<=50K,0
1,2a,NaN,Self-emp-not-inc,83311.0,4.920702,None,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,UnitedStates,<=50K,0
2,3a,38.0,Private,215646.0,5.333741,None,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,UnitedStates,<=50K,0
3,4a,53.0,Private,234721.0,5.370552,None,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,UnitedStates,<=50K,0
4,5a,NaN,Private,338409.0,5.529442,None,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K,0


# Attribute Binning (discretization)
- API specification of function attribute_binning can be found <a href="../api_specification/com/mw/ds/data_transformer/transformers.html#com.mw.ds.data_transformer.transformers.attribute_binning">here</a>
- Supports numerical attributes only. 2 binning options: Equal Range Binning and Equal Frequency Binning


In [4]:
from anovos.data_transformer.transformers import attribute_binning

In [5]:
# Example 1 - Equal Range Binning (each bin is of equal size/width) + append tranformed columns
cols = ["education-num", "hours-per-week"]
odf = attribute_binning(df.select(cols), list_of_cols=cols, method_type="equal_range", 
                        bin_size=5, output_mode="append", print_impact=True)

odf.toPandas().head(5)

+--------------------+-------------+
|           attribute|unique_values|
+--------------------+-------------+
|education-num_binned|            5|
|hours-per-week_bi...|            5|
+--------------------+-------------+



,education-num,hours-per-week,education-num_binned,hours-per-week_binned
0,13.0,40.0,4.0,3.0
1,13.0,13.0,4.0,1.0
2,9.0,40.0,3.0,3.0
3,7.0,40.0,2.0,3.0
4,13.0,40.0,4.0,3.0


In [6]:
# Distinct values after binning
odf.select('hours-per-week_binned').distinct().orderBy('hours-per-week_binned').toPandas().head(10)

,hours-per-week_binned
0,NaN
1,1.0
2,2.0
3,3.0
4,4.0
5,5.0


In [7]:
# Example 2 - Equal Frequency Binning (each bin has equal no. of rows); replace original columns by transformed ones
cols = ["education-num", "hours-per-week"]
odf = attribute_binning(df.select(cols), list_of_cols=cols, method_type="equal_frequency", 
                           bin_size=5, print_impact=True)

odf.toPandas().head(5)

+--------------+-------------+
|     attribute|unique_values|
+--------------+-------------+
| education-num|            4|
|hours-per-week|            4|
+--------------+-------------+



,education-num,hours-per-week
0,4.0,2.0
1,4.0,1.0
2,1.0,2.0
3,1.0,2.0
4,4.0,2.0


In [8]:
# Distinct values after binning
odf.select('hours-per-week').distinct().orderBy('hours-per-week').toPandas().head(10)

,hours-per-week
0,NaN
1,1.0
2,2.0
3,4.0
4,5.0


In [30]:
# Example 3 - Save binning model + Applying saved model
cols = ["education-num", "hours-per-week"]
odf = attribute_binning(df.select(cols), list_of_cols=cols, method_type="equal_frequency", 
                           bin_size=5, pre_existing_model=False, model_path=outputPath + "/attribute_binning")

odf = attribute_binning(df.select(cols), list_of_cols=cols, method_type="equal_frequency", 
                           bin_size=5, pre_existing_model=True, model_path=outputPath + "/attribute_binning")
odf.toPandas().head(5)

,education-num,hours-per-week
0,4.0,2.0
1,4.0,1.0
2,1.0,2.0
3,1.0,2.0
4,4.0,2.0


# Monotonic Binning
- API specification of function monotonic_binning can be found <a href="../api_specification/com/mw/ds/data_transformer/transformers.html#com.mw.ds.data_transformer.transformers.monotonic_binning">here</a>
- Bin size is computed dynamically

In [9]:
from anovos.data_transformer.transformers import monotonic_binning

In [10]:
# Example 1 - Equal Range Binning (each bin is of equal size)
cols = ["education-num", "hours-per-week"]
odf = monotonic_binning(df.select(cols+["income"]), list_of_cols=cols, label_col="income", 
                       event_label=">50K", bin_method="equal_range", output_mode="append")
odf.toPandas().head(5)

,education-num,hours-per-week,income,education-num_binned,hours-per-week_binned
0,13.0,40.0,<=50K,6.0,2.0
1,13.0,13.0,<=50K,6.0,1.0
2,9.0,40.0,<=50K,4.0,2.0
3,7.0,40.0,<=50K,3.0,2.0
4,13.0,40.0,<=50K,6.0,2.0


In [11]:
# Distinct values after binning
odf.select("education-num_binned").distinct().orderBy('education-num_binned').toPandas()

,education-num_binned
0,NaN
1,1.0
2,2.0
3,3.0
4,4.0
5,5.0
6,6.0
7,7.0


In [12]:
odf.select("hours-per-week_binned").distinct().orderBy('hours-per-week_binned').toPandas()

,hours-per-week_binned
0,NaN
1,1.0
2,2.0
3,3.0


In [13]:
# Example 2 - Equal Frequency Binning (each bin has equal no. of rows)
cols = ["education-num", "hours-per-week"]
odf = monotonic_binning(df.select(cols+["income"]), list_of_cols=cols, label_col="income", 
                       event_label=">50K", bin_method="equal_frequency", output_mode="append")
odf.toPandas().head(5)

,education-num,hours-per-week,income,education-num_binned,hours-per-week_binned
0,13.0,40.0,<=50K,12.0,2.0
1,13.0,13.0,<=50K,12.0,1.0
2,9.0,40.0,<=50K,3.0,2.0
3,7.0,40.0,<=50K,2.0,2.0
4,13.0,40.0,<=50K,12.0,2.0


In [14]:
# Distinct values after binning
odf.select("education-num_binned").distinct().orderBy('education-num_binned').toPandas()

,education-num_binned
0,NaN
1,1.0
2,2.0
3,3.0
4,7.0
5,11.0
6,12.0
7,14.0
8,15.0


In [15]:
odf.select("hours-per-week_binned").distinct().orderBy('hours-per-week_binned').toPandas()

,hours-per-week_binned
0,NaN
1,1.0
2,2.0
3,6.0
4,7.0


# Categorical Attribute to Numerical Attribute Conversion
- API specification of function cat_to_num_unsupervised can be found <a href="../api_specification/com/mw/ds/data_transformer/transformers.html#com.mw.ds.data_transformer.transformers.cat_to_num_unsupervised">here</a>
- Supports Label Encoding and One hot encoding

In [16]:
from anovos.data_transformer.transformers import cat_to_num_unsupervised

In [17]:
# Example 1 - Label Encoding
odf = cat_to_num_unsupervised (df.select('income'), list_of_cols='income', method_type=1, 
                               index_order='alphabetAsc', print_impact=True)

Before
+-------+------+
|summary|income|
+-------+------+
|  count| 32561|
|    min| <=50K|
|    max|  >50K|
+-------+------+

After
+-------+------+
|summary|income|
+-------+------+
|  count| 32561|
|    min|     0|
|    max|     1|
+-------+------+



In [18]:
# Example 2 - One hot encoding
odf = cat_to_num_unsupervised (df.select('relationship'), list_of_cols='relationship', 
                               method_type=0, index_order='frequencyDesc', print_impact=True)

Before
root
 |-- relationship: string (nullable = true)

After
root
 |-- relationship_0: integer (nullable = true)
 |-- relationship_1: integer (nullable = true)
 |-- relationship_2: integer (nullable = true)
 |-- relationship_3: integer (nullable = true)
 |-- relationship_4: integer (nullable = true)
 |-- relationship_5: integer (nullable = true)
 |-- relationship_6: integer (nullable = true)
 |-- relationship_7: integer (nullable = true)
 |-- relationship_8: integer (nullable = true)



In [19]:
odf.toPandas().head(5)

,relationship_0,relationship_1,relationship_2,relationship_3,relationship_4,relationship_5,relationship_6,relationship_7,relationship_8
0,0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0


# Missing Value Imputation
- API specification of function imputation_MMM can be found <a href="../api_specification/com/mw/ds/data_transformer/transformers.html#com.mw.ds.data_transformer.transformers.imputation_MMM">here</a>

In [20]:
from anovos.data_transformer.transformers import imputation_MMM

In [21]:
# Example 1 - numerical column 
cols = ["education-num", "hours-per-week"]
odf = imputation_MMM(df.select(cols), list_of_cols=cols, method_type="median", output_mode="append", print_impact=True)

+--------------+-------------------+--------------------+-------------+
|     attribute|missingCount_before|     attribute_after|missing_count|
+--------------+-------------------+--------------------+-------------+
|hours-per-week|                109|hours-per-week_im...|            0|
| education-num|                 31|education-num_imp...|            0|
+--------------+-------------------+--------------------+-------------+



In [22]:
odf.where((F.col("education-num").isNull())|(F.col("hours-per-week").isNull())).distinct().toPandas().head(5)

,education-num,hours-per-week,education-num_imputed,hours-per-week_imputed
0,NaN,45.0,10,45
1,10.0,NaN,10,40
2,13.0,NaN,13,40
3,11.0,NaN,11,40
4,NaN,60.0,10,60


In [23]:
# Example 2 - categorical column
cols = ['relationship', 'sex']
odf = imputation_MMM(df.select(cols), list_of_cols=cols, method_type="median", output_mode="append", print_impact=True)

+------------+-------------------+--------------------+-------------+
|   attribute|missingCount_before|     attribute_after|missing_count|
+------------+-------------------+--------------------+-------------+
|         sex|                  4|         sex_imputed|            0|
|relationship|                  4|relationship_imputed|            0|
+------------+-------------------+--------------------+-------------+



In [24]:
odf.where((F.col("relationship").isNull())|(F.col("sex").isNull())).toPandas().head(5)

,relationship,sex,relationship_imputed,sex_imputed
0,None,None,Husband,Male
1,None,None,Husband,Male
2,None,None,Husband,Male
3,None,None,Husband,Male


# Outlier Categories Treatment
- API specification of function outlier_categories can be found <a href="../api_specification/com/mw/ds/data_transformer/transformers.html#com.mw.ds.data_transformer.transformers.outlier_categories">here</a>
- Supports 2 ways of outliers detection: by max number of categories and by coverage (%)

In [25]:
from anovos.data_transformer.transformers import outlier_categories

In [26]:
# Example 1 - Max N categories
cols = ['education', 'occupation']
odf = outlier_categories(df.select(cols), list_of_cols=cols, max_category=5, 
                                          output_mode='append', print_impact=True)

odf.select('education', 'education_outliered').orderBy('education_outliered').distinct().toPandas()

+----------+-------------------+
| attribute|uniqueValues_before|
+----------+-------------------+
| education|                 16|
|occupation|                 15|
+----------+-------------------+

+--------------------+------------------+
|           attribute|uniqueValues_after|
+--------------------+------------------+
| education_outliered|                 5|
|occupation_outliered|                 5|
+--------------------+------------------+



,education,education_outliered
0,None,None
1,Bachelors,Bachelors
2,HS-grad,HS-grad
3,Masters,Masters
4,Some-college,Some-college
5,11th,others
6,Assoc-acdm,others
7,Assoc-voc,others
8,?,others
9,Doctorate,others


In [27]:
# Example 2 - by coverage (top frequently seen categories are considered till it covers minimum N% of rows)
cols = ['education', 'occupation']
odf = outlier_categories(df.select(cols), list_of_cols=cols, coverage=0.9, 
                                          output_mode='append', print_impact=True)

odf.select('education', 'education_outliered').orderBy('education_outliered').distinct().toPandas()

+----------+-------------------+
| attribute|uniqueValues_before|
+----------+-------------------+
| education|                 16|
|occupation|                 15|
+----------+-------------------+

+--------------------+------------------+
|           attribute|uniqueValues_after|
+--------------------+------------------+
| education_outliered|                 9|
|occupation_outliered|                11|
+--------------------+------------------+



,education,education_outliered
0,None,None
1,10th,10th
2,11th,11th
3,Assoc-acdm,Assoc-acdm
4,Assoc-voc,Assoc-voc
5,Bachelors,Bachelors
6,HS-grad,HS-grad
7,Masters,Masters
8,Some-college,Some-college
9,?,others
